In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
#from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (8/Mar/2023)
v18.14.2


# Single Trades [NB31]

_allows to choose a market, retrieves market information, and then submits a trade_

In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=False, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
SDK.version().get("msg")

'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(index=1, options=(('USDC/USDT', 'USDC/USDT'), ('USDC/ETH', 'USDC/ETH'), ('BNT/USDT', 'BNT/USDT'), ('B…

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [6]:
m = SDK.mGetMarketByPair(pair=pair)
m

CarbonSDK.MarketByPair(pair='ETH/USDC', bestBid=1599.9999998788553, bestAsk=1630.0000000000148, liqBid=2499052.640732, liqAsk=2161.9733078395298, lastBid=799.9999998679182, lastAsk=3000.0000000000327)

In [7]:
print(f"bid/mid/ask = {SDK.roundsd(m.bestBid,6)}/{SDK.roundsd(m.mid,6)}/{SDK.roundsd(m.bestAsk,6)}")
print(f"liquidity = {SDK.roundsd(m.liqBid,4)} {m.liqBidUnit} bid, {SDK.roundsd(m.liqAsk,4)} {m.liqAskUnit} ask")

bid/mid/ask = 1600.0/1615.0/1630.0
liquidity = 2499100.0 USDC bid, 2162.0 ETH ask


## Trading settings

In [8]:
m.cpair

P('ETH/USDC')

In [9]:
try:
    bq_w()
except:
    bq_w = DropdownManager(["Base Token", "Quote Token"], defaultval="Base Token")
    bq_w()
tkn = m.cpair.tknb if bq_w.value == "Base Token" else m.cpair.tknq

Dropdown(options=(('Base Token', 'Base Token'), ('Quote Token', 'Quote Token')), value='Base Token')

In [10]:
try:
    buysell_w()
except:
    buysell_w = DropdownManager(["Buy", "Sell"], defaultval="Sell")
    buysell_w()
buysell = SDK.SELL if buysell_w.value == "Sell" else SDK.BUY

Dropdown(index=1, options=(('Buy', 'Buy'), ('Sell', 'Sell')), value='Sell')

In [11]:
print(f"Your choice: {buysell_w.value} {tkn} against {m.cpair.other(tkn)} [market = {m.pair}]")

Your choice: Sell ETH against USDC [market = ETH/USDC]


In [12]:
amount = 1000

#### `mPrepareTrade`

In [13]:
trade = SDK.mPrepareTrade(pair, buysell, tkn, amount)
trade

CarbonSDK.TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='SELL', amount=1000, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 487, 'amount': 322748612218223762}, {'strategyId': 488, 'amount': 322748612218223762}, {'strategyId': 489, 'amount': 322748612218223762}, {'strategyId': 490, 'amount': 322748612218223762}, {'strategyId': 9, 'amount': 998709005551127104952}], actionsWei=[{'id': 487, 'input': 322748612218223762, 'output': 500000000}, {'id': 488, 'input': 322748612218223762, 'output': 500000000}, {'id': 489, 'input': 322748612218223762, 'output': 500000000}, {'id': 490, 'input': 322748612218223762, 'output': 500000000}, {'id': 9, 'input': 998709005551127104952, 'output': 1235489619719}], actionsTokenRes=[{'id': 487, 'sourceAmount': 0.3227486122

In [14]:
trade.actionsTokenRes

[{'id': 487, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0},
 {'id': 488, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0},
 {'id': 489, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0},
 {'id': 490, 'sourceAmount': 0.32274861221822376, 'targetAmount': 500.0},
 {'id': 9, 'sourceAmount': 998.7090055511271, 'targetAmount': 1235489.619719}]

#### `mComposeTradeTransaction`

In [15]:
tx = SDK.mComposeTradeTransaction(**trade.cttkwargs)
tx

{'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000102ec8a4428000000000000000000000000000000000000000000000000000000000000000500000000000000000000000000000000000000000000000000000000000001e7000000000000000000000000000000000000000000000000047aa2271f047c9200000000000000000000000000000000000000000000000000000000000001e8000000000000000000000000000000000000000000000000047aa2271f047c9200000000000000000000000000000000000000000000000000000000000001e9000000000000000000000000000000000000000000000000047aa2271f047c9200000000000000000000000000000000000000000000000000000000000001ea000000000000000000000000000000000000000000000000047aa2271f047c92000000000000000000000000000000000000000000000000000000000000000900000000000000000000

#### `signsubmittx`

In [16]:
try:
    submit_w()
except:
    submit_w = CheckboxManager(["submit"], values=[0])
    submit_w()

In [17]:
if submit_w.values[0]: 
    r = SDK.signsubmittx(tx, sign=True)
else:
    r = "check box above to submit"
r

'check box above to submit'